In [ ]:
!pip install --quiet optuna

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
drive.mount('/content/drive')

import optuna

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
# Load the Dataset
deal = pd.read_csv('/content/drive/MyDrive/데청캠_인바디/ZohoCRM_Dataset/deal_closed_prediction_data.csv')

print(deal)

X = deal[['Year', 'Month', 'Day', 'LeadSource', 'Territory', 'Industry']]
y = deal[['Stage']]

print(y.value_counts())

# StandarScaler 적용
# scaler = StandardScaler()
# scaler.fit(X)
# X_scaled = scaler.transform(X)

# Min-Max Scaling
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(X) 
scaled = scaler.transform(X) 

      Year  Month  Day  LeadSource    Territory       Industry  Stage
0     2017      2    7    0.000458    50.000000  132320.620804      0
1     2017      2    7    0.000458    50.000000  132320.620804      0
2     2017      2    7    0.000458  3430.305858  132320.620804      1
3     2017      2    7    0.000458  1394.639984   61963.087644      1
4     2017      2   14    0.000458    50.000000  132320.620804      1
...    ...    ...  ...         ...          ...            ...    ...
2136  2021      7   24   32.692308   645.756033    1430.564418      0
2137  2021      7   26   25.000000   645.756033    1430.564418      0
2138  2021      7   26   25.000000  1014.059096    2727.317659      0
2139  2021      7   26   40.384615   645.756033    2727.317659      0
2140  2021      7   26   32.692308   645.756033    1073.495543      0

[2141 rows x 7 columns]
Stage
0        1212
1         929
dtype: int64


In [49]:
# Split the Dataset
X_train, X_test, y_train, y_test = train_test_split(scaled , y, test_size=0.3, random_state=1)

print(X_train.shape)
print(y_test.shape)

(1498, 6)
(643, 1)


In [50]:
import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
import sklearn.svm


def objective(trial):
    
    c = trial.suggest_float('svc_c', 1e-2, 1e6, log=True)
    gam = trial.suggest_float('svc_gamma', 1e-3, 1e6, log=True)
    
    clf = sklearn.svm.SVC (
        kernel='rbf', C=c, gamma=gam
        )
    
    return sklearn.model_selection.cross_val_score(
        clf, X_train, y_train, n_jobs=-1, cv=5).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

# Accuracy: 0.8424593088071349
# Best hyperparameters: {'svc_c': 1.3720361031797952, 'svc_gamma': 6.033860355438825}
# Accuracy: 0.8411237458193981
# Best hyperparameters: {'svc_c': 1.0050140664400131, 'svc_gamma': 13.031717251398376}
# Accuracy: 0.8437948717948718
# Best hyperparameters: {'svc_c': 2.206225165264988, 'svc_gamma': 4.270221263739784}

[I 2021-08-27 16:25:29,055] A new study created in memory with name: no-name-e9753db8-0bfe-43e9-a1ec-e528ae99ecea
[I 2021-08-27 16:25:30,679] Trial 0 finished with value: 0.6575429208472686 and parameters: {'svc_c': 0.7524810919944533, 'svc_gamma': 834505.5400358414}. Best is trial 0 with value: 0.6575429208472686.
[I 2021-08-27 16:25:31,089] Trial 1 finished with value: 0.6688851727982164 and parameters: {'svc_c': 4.476169437043126, 'svc_gamma': 2336.832464084838}. Best is trial 1 with value: 0.6688851727982164.
[I 2021-08-27 16:25:31,370] Trial 2 finished with value: 0.7930434782608696 and parameters: {'svc_c': 0.49861566782273326, 'svc_gamma': 0.13454088031911673}. Best is trial 2 with value: 0.7930434782608696.
[I 2021-08-27 16:25:31,847] Trial 3 finished with value: 0.6788896321070235 and parameters: {'svc_c': 16.078701941686894, 'svc_gamma': 1000.9927090641468}. Best is trial 2 with value: 0.7930434782608696.
[I 2021-08-27 16:25:32,251] Trial 4 finished with value: 0.666218506131

Accuracy: 0.8424637681159421
Best hyperparameters: {'svc_c': 2.576074542111366, 'svc_gamma': 4.465364340220218}


In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
clf = sklearn.svm.SVC (kernel='rbf', C=2.206225165264988,gamma=4.270221263739784)
clf.fit(X_train,y_train)
pred_y = clf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [ ]:
# 혼동행렬, 정확도, 정밀도, 재현율, F1, AUC 불러오기
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))

# 0.7107
# 0.8289
# 0.8398
# 0.8429
# 0.8445
#0.8445
#0.8367
get_clf_eval(y_test, pred_y)

오차행렬:
 [[335  31]
 [ 69 208]]

정확도: 0.8445
정밀도: 0.8703
재현율: 0.7509
F1: 0.8062
AUC: 0.8331
